## Working with multiple objects in the Python SDK
AIS supports multi-object operations on groups of objects. An `ObjectGroup` can be created with one of:
- a list of object names
- an [ObjectRange](https://github.com/NVIDIA/aistore/blob/master/python/aistore/sdk/object_range.py)
- a string template.

In [ ]:
pip install aistore

### Set up the client and create necessary buckets

In [ ]:
from aistore import Client
from aistore.sdk.errors import AISError
import os

ais_url = os.getenv("AIS_ENDPOINT", "http://localhost:8080")
client = Client(ais_url)
bucket = client.bucket("my-bck").create(exist_ok=True)
copy_dest_bucket = client.bucket("copy-destination-bucket").create(exist_ok=True)
transform_dest_bucket = client.bucket("transform-destination-bucket").create(
    exist_ok=True
)

### Create some objects in the bucket

In [ ]:
object_names = [f"example_obj_{i}" for i in range(10)]
for name in object_names:
    bucket.object(name).put_content("object content".encode("utf-8"))

### Create Object Group by list of names

In [ ]:
my_objects = bucket.objects(obj_names=object_names)

### Create Object Group by ObjectRange

In [ ]:
from aistore.sdk.multiobj import ObjectRange

my_object_range = ObjectRange(prefix="example_obj_", min_index=1, max_index=3)
my_objects = bucket.objects(obj_range=my_object_range)

### Create Object Group by Template String
String templates can be passed directly to AIS following the [syntax described here](https://github.com/NVIDIA/aistore/blob/master/docs/batch.md#operations-on-multiple-selected-objects)

In [ ]:
# Equivalent to the range above
my_object_template = "example_obj_{1..3}"
my_objects = bucket.objects(obj_template=my_object_template)
# More advanced template example with multiple ranges and defined steps
complex_range = "example_obj_{0..10..2}_details_{1..9..2}.file-extension"

### Prefetch or evict multiple objects when using a bucket with a cloud backend

In [ ]:
my_objects.prefetch()
my_objects.evict()

### Copy multiple objects

Copies selected objects directly to the new bucket

In [ ]:
copy_job = my_objects.copy(to_bck=copy_dest_bucket)
# The job will reach an idle state before finishing, so wait for idle
client.job(job_id=copy_job).wait_for_idle()
# See the objects in the destination bucket
copy_dest_bucket.list_all_objects()

### Delete multiple objects from the destination bucket above

In [ ]:
all_objects = copy_dest_bucket.list_all_objects()
# Creates a group including all objects from the destination bucket
objects_to_delete = copy_dest_bucket.objects(
    obj_names=[entry.name for entry in all_objects]
)
delete_job_id = objects_to_delete.delete()
client.job(delete_job_id).wait()
after_deletion = copy_dest_bucket.list_all_objects()
print(
    f"Objects before deletion: {len(all_objects)}, objects after deletion: {len(after_deletion)}"
)

#### Transform -- Provide an ETL to be performed on each object so the result appears in the destination bucket.

Note: This step requires the AIS cluster to be running in Kubernetes; see [getting_started](https://github.com/NVIDIA/aistore/blob/master/docs/getting_started.md#kubernetes-playground) for setup info.

In [ ]:
# First create an ETL
# This is a simple example transform that reverses each object's contents (assuming utf-8 encoded text)
def transform(input_bytes):
    reversed_in_str = input_bytes.decode("utf-8")[::-1]
    return reversed_in_str.encode()


etl_name = "multiobj-transform-example"
try:
    client.etl().init_code(transform=transform, etl_name=etl_name)
except AISError as err:
    print(err)

# Now run the transform with the etl name specified
transform_job = my_objects.transform(etl_name=etl_name, to_bck=transform_dest_bucket)
client.job(job_id=transform_job).wait_for_idle()

# The output will be in the destination bucket
transformed_objs = transform_dest_bucket.list_all_objects()

# See the result
for entry in transformed_objs:
    input_data = bucket.object(entry.name).get().read_all()
    output_data = transform_dest_bucket.object(entry.name).get().read_all()
    print(f"Object {entry.name} {input_data} => {output_data}")

### Cleanup buckets

In [ ]:
for bck in [bucket, copy_dest_bucket, transform_dest_bucket]:
    bck.delete(missing_ok=True)